# Chapter 3 - It starts with a tensor

Tensors are the fundamental data structure in PyTorch. They are multidimensional arrays that stores a collection of numbers. These numbers are accessible individually using an index.

In [1]:
import torch
a = torch.ones(3) # Creates a one-dimensional tensor of size 3 filled with 1s.
a

tensor([1., 1., 1.])

In [2]:
a[1] # Accesses second element of tensor.

tensor(1.)

In [3]:
float(a[1]) # Returns second element of tensor as float.

1.0

In [4]:
a[2] = 2.0 # Changes third element of tensor to 2.
a

tensor([1., 1., 2.])

## 1 - 2D tensors

To store multiple coordinates in a tensor, we can do:

In [5]:
points = torch.zeros(6)
points[0] = 4.0
points[1] = 1.0
points[2] = 5.0
points[3] = 3.0
points[4] = 2.0
points[5] = 1.0

Alternatively:

In [6]:
points = torch.tensor([4.0, 1.0, 5.0, 3.0, 2.0, 1.0]) # Creates tensor by passing list to constructor.
points

tensor([4., 1., 5., 3., 2., 1.])

To get the coordinates of the first point:

In [7]:
float(points[0]), float(points[1])

(4.0, 1.0)

However, this is impractical. It is more efficient to use a 2D tensor:

In [8]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]]) # Creates 2D tensor.
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [9]:
points.shape # Returns shape of tensor

torch.Size([3, 2])

In [10]:
points[0, 1] # Returns y-coordinate of first point.

tensor(1.)

In [11]:
points[0] # Returns coordinate of first point.

tensor([4., 1.])

Tensors can be range indexed just like Python lists:

In [12]:
points[1:]

tensor([[5., 3.],
        [2., 1.]])

In [13]:
points[1:, :]

tensor([[5., 3.],
        [2., 1.]])

In [14]:
points[1:, 0]

tensor([5., 2.])

In [15]:
points[None] # Adds a dimension of size 1.

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

## 2 - Named tensors

(Note: named tensors are still an experimental feature at the time of writing)

The dimensions of tensors usually index things like pixel locations or color channels. When the tensor gets transformed, keeping track of which dimension contains what data can be error-prone. The `tensor` function takes a `names` argument (a sequence of strings):

In [16]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

/tmp/ipykernel_7999/2371314847.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1463.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

If a tensor already exists and want to add names to it, we can use the `refine_names` method:

In [17]:
img_t = torch.randn(3, 5, 5)
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
print('img named:', img_named.shape, img_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')


The `align_as` method returns a tensor with the missing dimensions added and existing ones permuted to the right order:

In [18]:
print('weights named:', weights_named.shape, weights_named.names)
print('img named:', img_named.shape, img_named.names)

weights_aligned = weights_named.align_as(img_named)
print('weights aligned:', weights_aligned.shape, weights_aligned.names)

weights named: torch.Size([3]) ('channels',)
img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
weights aligned: torch.Size([3, 1, 1]) ('channels', 'rows', 'columns')


Functions like `sum`, which accept a dimension argument, can also take named dimension:

In [19]:
gray_named = (img_named*weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

Combining tensors that have dimensions with different names gives an error. For example, if we run the following line of code:

```
gray_named = (img_named[..., :3]*weights_named).sum('channels')
```

it would give an error since:

In [20]:
print(img_named[..., :3].names)
print(weights_named.names)

('channels', 'rows', 'columns')
('channels',)


If we want to make a named tensor unnamed:

In [21]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

## 3 - More on tensors

The `dtype` argument for tensors specifies the data type of elements that will be contained in the tensor. The following are the possible values for the `dtype` argument:

- `torch.float32` or `torch.float`
- `torch.float64` or `torch.double`
- `torch.float16` or `torch.half`
- `torch.int8`
- `torch.uint8`
- `torch.int16` or `torch.short`
- `torch.int32` or `torch.int`
- `torch.int64` or `torch.long`
- `torch.bool`

The `dtype` argument can be specified as follows:

In [22]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)

short_points.dtype # Returns the data type of the tensor short_points

torch.int16

The output of a tensor creation function can also be casted to the desired data type:

In [23]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

Alternatively:

In [24]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)

When mixing input types, the inputs are converted to the larger data type:

In [25]:
points_64 = torch.rand(5, dtype=torch.double)
points_short = points_64.to(torch.short)
points_64 * points_short

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

## 4 - Tensors as scenic view of storage

Values in tensors are allocated in contiguous chunks of memory managed by `tensor.Storage` instances. The following shows indexing into storage works with 2D tensors:

In [26]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage() # Accesses the storage of the points tensor

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

The storages can be indexed manually as well:

In [27]:
points.storage()[0]

4.0

In addition, changing the value of a storage leads to changing the content of its referring tensor:

In [28]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

Some methods of the `Tensor` object operates *in place* by modifying the input instead of creating and returning a new tensor object. They are recognizable by the trailing underscore in their name:

In [29]:
a = torch.ones(3, 2)
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

## 5 - Tensor metadata

Tensors rely on a few pieces of information in order to index into a storage: size, offset and stride.
- Size: Indicates how many elements there are in each dimension
- Offset: Index in the storage referring to the first element of the tensor
- Stride: Number of elements to skip in order to obtain the next element in each dimension.

In [30]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
print('Size:', points.size()) # This contains the same information as points.shape.
print('Offset:', points.storage_offset())
print('Stride:', points.stride())

Size: torch.Size([3, 2])
Offset: 0
Stride: (2, 1)


This relationship between tensors and storages mean that operations like tranpose and extracting subtensors are inexpensive since memory reallocation is not required. Instead, a new tensor object is allocated with different values for size, offset and stride.

The following extracts a subtensor from the `points` tensor and displays the metadata:

In [31]:
second_point = points[1]
print('Size:', second_point.size())
print('Offset:', second_point.storage_offset())
print('Stride:', second_point.stride())

Size: torch.Size([2])
Offset: 2
Stride: (1,)


`points` and `second_point` share the same storage object:

In [32]:
points.storage().data_ptr() == second_point.storage().data_ptr()

True

Hence, changing the subtensor also affects the original tensor:

In [33]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In cases where this is not desirable, we can clone the subtensor:

In [34]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

Note that `second_point` does not share its storage object with `points` anymore:

In [35]:
second_point.storage().data_ptr() == points.storage().data_ptr()

False

The same idea applies for transposing tensors:

In [36]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

`points` and `points_t` share the same storage:

In [37]:
points.storage().data_ptr() == points_t.storage().data_ptr()

True

The only difference between the two tensors is their shape and stride

In [38]:
print(points.stride())
print(points_t.stride())

(2, 1)
(1, 2)


Also, while `points` is a contiguous tensor, `points_t` is not. A contiguous tensor is one such that the values are laid out in the storage by starting from the right most dimension. 

In [39]:
print(points)
print(points.storage())
print(points.is_contiguous())

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]
True


In [40]:
print(points_t)
print(points_t.storage())
print(points_t.is_contiguous())

tensor([[4., 5., 2.],
        [1., 3., 1.]])
 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]
False


## 6 - Moving tensors to the GPU

Tensors have an attribute called `device`. To create a tensor in the GPU:

In [41]:
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device='cuda')
points_gpu.device

device(type='cuda', index=0)

To move an existing tensor to the GPU:

In [42]:
points_gpu = points.to(device='cuda')
points_gpu.device

device(type='cuda', index=0)

If there was more than one GPU, the specific choice of GPU can be assigned as well by passing a zero-based integer:

In [43]:
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device='cuda:0')

All calculations for tensors in the GPU are kept in the GPU:

In [44]:
points_gpu = points_gpu + 4
points_gpu.device

device(type='cuda', index=0)

To move a tensor back to the CPU:

In [45]:
points_cpu = points_gpu.to(device='cpu')
points_cpu.device

device(type='cpu')

Shorthand methods also exist:

In [46]:
points_gpu = points.cuda() # Dedfaults to GPU index 0
points_gpu = points.cuda(0)
points_cpu = points.cpu()

## 7 - Tensors and NumPy

To convert a PyTorch tensor to a NumPy array:

In [47]:
points = torch.ones(3, 4)
points_np = points.numpy()
points_np

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

Conversely, a NumPy array can be converted to a PyTorch tensor:

In [48]:
points = torch.from_numpy(points_np)
points

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

As a side note, the default numeric type in PyTorch is 32-bit floating-point while for NumPy it is 64-bit.

## 8 - Serializing tensors

The following code serializes a tensor (PyTorch uses `pickle`):

In [49]:
torch.save(points, './data/ch3/ourpoints.t')

Alternatively:

In [50]:
with open('./data/ch3/ourpoints.t', 'wb') as f:
    torch.save(points, f)

To load the tensor back to the program:

In [51]:
points = torch.load('./data/ch3/ourpoints.t')

Alternatively,

In [52]:
with open('./data/ch3/ourpoints.t', 'rb') as f:
    points = torch.load(f)

Instead of pickle, the HDF5 format can be used through the `h5py` library:

In [53]:
import h5py

f = h5py.File('./data/ch3/ourpoints.hdf5', 'w')
dset = f.create_dataset('coords', data=points.numpy()) # 'coords' is a key into the HDF5 file.
f.close()

To load the HDF5 file into the program:

In [54]:
f = h5py.File('./data/ch3/ourpoints.hdf5', 'r')
dset = f['coords'] # Returns a dataset
points = torch.from_numpy(dset[...])
f.close()

points

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])